In [1]:
import numpy as np
import time
import pickle
from numpy.random import uniform
import matplotlib.pyplot as plt

In [100]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None, action=None):
        self.parent = parent
        self.position = position
        self.action = action

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def get_closest_grid(current_pos):
    differences = current_pos-uncertainty_grids
    distances = np.sum(differences*differences,axis=1)
    min_ind = np.argmin(distances)
#     print ("read pos: {0} index: {1} found_pos: {2}".format(current_pos, min_ind, uncertainty_grids[min_ind]))
    
    return min_ind

def get_obstacle_indices():
    grid_res = 1.0
    obstacle_indices = []
    obstacle_indices_unsquezed = []
    
    obstacle_start = np.array([[-20, -20], [-20, -20], [19, -20], [-20, 19]]) 
    obstacle_end = np.array([[-19, 20], [20, -19], [20, 20], [20, 20]])

    for i in range(obstacle_start.shape[0]):
        x_range = np.arange(-grid_res/2+obstacle_start[i,0], obstacle_end[i,0]+grid_res/2, grid_res/4)
        y_range = np.arange(-grid_res/2+obstacle_start[i,1], obstacle_end[i,1]+grid_res/2, grid_res/4)
        indices = []
        for x in x_range:
            for y in y_range:
                current_pos = np.array([x,y])
                current_ind = get_closest_grid(current_pos)
                if current_ind not in indices:
                    indices.append(current_ind)

        obstacle_indices.append(indices)
        
    for i in range(len(obstacle_indices)):
        for j in range(len(obstacle_indices[0])):
            obstacle_indices_unsquezed.append(obstacle_indices[i][j])

    return obstacle_indices_unsquezed

In [105]:
#Grid information
grid_res = 2.0
x_lim, y_lim, z_lim = 20, 20, 6

def astar_drone(start, end, obstacle_indices=None):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0
    visited_grids = []

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:
        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append([current.position, current.action])
                visited_grids.append(get_closest_grid(current.position))
                current = current.parent
            return path[::-1], visited_grids # Return reversed path and visited grids

        # Generate children
        children = []
        for index, new_position in enumerate([(0, -grid_res), (0, grid_res), (-grid_res, 0), (grid_res, 0)]): # Adjacent squares
#         for new_position in [(0, -grid_res, 0), (0, grid_res, 0), (-grid_res, 0, 0), (grid_res, 0, 0)]: # Adjacent squares
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
#             print ("Node position: ", node_position)
            node_index = get_closest_grid(node_position)
#             print ("Node index: {0} Node pos: {1}/{2}".format(node_index, uncertainty_grids[node_index], node_position))
            
            
            # Make sure within range
            if node_position[0] > x_lim or node_position[0] < -x_lim or node_position[1] > y_lim or node_position[1] < -y_lim:
#                 print ("It's not within the range. Node position: ", node_position)
                continue
              
                
            if node_index in obstacle_indices:
#                 print ("It's a obstacle place. Node position: ", node_position)
                continue
                    
                

            # Create new node
            new_node = Node(current_node, node_position, index)
            
            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue
            # Add the child to the open list
            open_list.append(child)

In [106]:
import time 

x_lim, y_lim, z_lim = 20, 20, 6
eps = 0.1
res = 2
X,Y = np.mgrid[-x_lim : x_lim + eps:res, 
               -y_lim : y_lim + eps:res]
uncertainty_grids = np.vstack((X.flatten(), Y.flatten())).T
    
t = time.time()


start = (12., 16.)
end = (12.,  10.)

path = astar_drone(start, end, obstacle_indices=obstacle_indices)
print(path)
elapsed = time.time() - t

([[(12.0, 16.0), None], [(12.0, 14.0), 0], [(12.0, 12.0), 0], [(12.0, 10.0), 0]], [351, 352, 353, 354])


In [62]:
obstacle_indices = get_obstacle_indices()

In [67]:
uncertainty_grids[obstacle_indices]

array([[-20., -20.],
       [-20., -19.],
       [-20., -18.],
       [-20., -17.],
       [-20., -16.],
       [-20., -15.],
       [-20., -14.],
       [-20., -13.],
       [-20., -12.],
       [-20., -11.],
       [-20., -10.],
       [-20.,  -9.],
       [-20.,  -8.],
       [-20.,  -7.],
       [-20.,  -6.],
       [-20.,  -5.],
       [-20.,  -4.],
       [-20.,  -3.],
       [-20.,  -2.],
       [-20.,  -1.],
       [-20.,   0.],
       [-20.,   1.],
       [-20.,   2.],
       [-20.,   3.],
       [-20.,   4.],
       [-20.,   5.],
       [-20.,   6.],
       [-20.,   7.],
       [-20.,   8.],
       [-20.,   9.],
       [-20.,  10.],
       [-20.,  11.],
       [-20.,  12.],
       [-20.,  13.],
       [-20.,  14.],
       [-20.,  15.],
       [-20.,  16.],
       [-20.,  17.],
       [-20.,  18.],
       [-20.,  19.],
       [-20.,  20.],
       [-19., -20.],
       [-19., -19.],
       [-19., -18.],
       [-19., -17.],
       [-19., -16.],
       [-19., -15.],
       [-19.,

In [92]:
init_range = np.arange(-18, 0, 2)

In [93]:
np.random.choice(init_range, (5,), replace=False)

array([ -2, -16, -10, -18,  -4])

In [125]:
map_lim = 20.0 - 2.0
init_pos_list = []
init_grid_list = []
for i in range(5):
    init_grid_list.append(get_closest_grid([-map_lim + 2*i, -map_lim]))
    init_pos_list.append([-map_lim + 2*i, -map_lim])
for i in range(1,6):
    init_grid_list.append(get_closest_grid([-map_lim , -map_lim + 2*i]))
    init_pos_list.append([-map_lim , -map_lim + 2*i])

In [126]:
np.random.choice(range(10),(5,),replace=False)

array([1, 3, 2, 9, 8])

In [148]:
# a = [[1,2,3,4],[5,6],[7,8,9,10,11],[15,25,24]]
for i in range(len(a)):
    if np.any(a[i]):
        print (a[i][0])
        del(a[i][0])

In [5]:
import operator

In [7]:
with open('model_reward_list.pkl', 'rb') as f:  
    model_reward_list = pickle.load(f)
model_reward_list = sorted(model_reward_list.items(), key=operator.itemgetter(1))
model_reward_list[-1]

(221000, 48.94)